In [126]:
#tensorflow 2.2.2
#others newest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

import tensorflow as tf
from glob import glob
import pickle
import numpy as np
import random
import os
import tqdm.keras as tk
from keras_tqdm import TQDMNotebookCallback
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [1]:
new_path = "./proj/new_train.nosync/"
test_path = "./proj/new_val_in.nosync/"

In [2]:
def train_generator_x(training_list, batch_size):
    index = 0
    print(len(training_list))
    while True:
        train_x = np.empty((batch_size,19))
        train_y = np.empty((batch_size,30))

        for i in range(batch_size):
            train_path = training_list[index]
            with open(train_path, 'rb') as f:
                data = pickle.load(f)                
                agent_id = data['agent_id']
                idx = np.where(data["track_id"] == data["agent_id"])[0][0]
                input_data = data['p_in'][idx,:,:]                
                input_data1 = input_data[:,0]
                train_x[i,:] = input_data1
                
                output_data = data['p_out'][idx,:,:]
                output_data1 = output_data[:,0]
                train_y[i,:] = output_data1

            index += 1
        if index == len(training_list):
            index = 0

        yield train_x, train_y

def valid_generator_x(valid_list, batch_size):
    print("we are in vad")
    index = 0
    while True:
        valid_x = np.empty((batch_size,19))
        valid_y = np.empty((batch_size,30))
        for i in range(batch_size):
            valid_path = valid_list[index]
            with open(valid_path, 'rb') as f:
                data = pickle.load(f)
                agent_id = data['agent_id']
                idx = np.where(data["track_id"] == data["agent_id"])[0][0]
                input_data = data['p_in'][idx,:,:]
                input_data1 = input_data[:,0]
                valid_x[i,:] = input_data1
                
                output_data = data['p_out'][idx,:,:]
                output_data1 = output_data[:,0]
                valid_y[i,:] = output_data1
            index += 1
        if index == len(valid_list):
            index = 0
        yield valid_x, valid_y

In [3]:
def train_generator_y(training_list, batch_size):
    index = 0
    print(len(training_list))
    while True:
        train_x = np.empty((batch_size,19))
        train_y = np.empty((batch_size,30))

        for i in range(batch_size):            
            train_path = training_list[index]
            with open(train_path, 'rb') as f:
                data = pickle.load(f)                
                agent_id = data['agent_id']
                idx = np.where(data["track_id"] == data["agent_id"])[0][0]
                input_data = data['p_in'][idx,:,:]                
                input_data1 = input_data[:,1]
                train_x[i,:] = input_data1
    
                output_data = data['p_out'][idx,:,:]
                output_data1 = output_data[:,1]
                train_y[i,:] = output_data1

            index += 1
        if index == len(training_list):
            index = 0
#         print("index is ", index)
#         print(train_x.shape)
        yield train_x, train_y

def valid_generator_y(valid_list, batch_size):
    print("we are in vad")
    index = 0
    while True:
        valid_x = np.empty((batch_size,19))
        valid_y = np.empty((batch_size,30))
        for i in range(batch_size):
            valid_path = valid_list[index]
            with open(valid_path, 'rb') as f:
                data = pickle.load(f)
                agent_id = data['agent_id']
                idx = np.where(data["track_id"] == data["agent_id"])[0][0]
                input_data = data['p_in'][idx,:,:]
                input_data1 = input_data[:,1]
                valid_x[i,:] = input_data1
                
                output_data = data['p_out'][idx,:,:]
                output_data1 = output_data[:,1]
                valid_y[i,:] = output_data1
            index += 1
        if index == len(valid_list):
            index = 0
        yield valid_x, valid_y

In [130]:
data_list = glob(os.path.join(train_path, '*'))
random.shuffle(data_list)
print(int(len(data_list)*1))
data_list = data_list[:len(data_list)]
print(int(len(data_list)*1))

train_list = data_list[:int(len(data_list)*0.9)-47]
valid_list = data_list[int(len(data_list)*0.9)+95:]
print(len(train_list))
print(len(valid_list))

205942
205942
185300
20500


In [131]:
#Model structure
#input shape (38) = 19x2
#output shape(60) = 30x2

model_x = Sequential()
model_x.add(Input(shape=(19,)))
model_x.add(Dense(units=512, activation='relu'))
model_x.add(Dropout(0.1))
model_x.add(Dense(units=512, activation='relu'))
model_x.add(Dropout(0.1))

model_x.add(Dense(units=30, activation='linear'))

model_y = Sequential()
model_y.add(Input(shape=(19,)))
model_y.add(Dense(units=512, activation='relu'))
model_y.add(Dropout(0.1))
model_y.add(Dense(units=512, activation='relu'))
model_y.add(Dropout(0.1))
model_y.add(Dense(units=30, activation='linear'))

In [132]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model_x.compile(loss=root_mean_squared_error, optimizer='adam')
checkpointer_x = ModelCheckpoint(filepath='./liner_regressor_x_weights.hdf5', verbose=2, save_best_only=True, mode='min')



In [133]:
model_y.compile(loss=root_mean_squared_error, optimizer='adam')
checkpointer_y = ModelCheckpoint(filepath='./liner_regressor_y_weights.hdf5', verbose=2, save_best_only=True, mode='min')

In [134]:
print('training...')
num_train = len(train_list)/100
num_test = len(valid_list)/100
train_gen_x = train_generator_x(train_list,100)
valid_gen_x = valid_generator_x(valid_list,100)

train_gen_y = train_generator_y(train_list,100)
valid_gen_y = valid_generator_y(valid_list,100)

training...


In [136]:
hist_x=model_x.fit(train_gen_x,
                        verbose=2,
                        epochs=1,
                        validation_data=valid_gen_x,
                        steps_per_epoch=(len(train_list)/100),
                        validation_steps=(len(valid_list)/100),
                        callbacks=[checkpointer_x, TQDMNotebookCallback()]
                        )


In [137]:
hist_y=model_y.fit(train_gen_y,
                        verbose=2,
                        epochs=1,
                        validation_data=valid_gen_y,
                        steps_per_epoch=(len(train_list)/100),
                        validation_steps=(len(valid_list)/100),
                        callbacks=[checkpointer_y, TQDMNotebookCallback()]
                        )

185300


we are in vad

Epoch 00001: val_loss improved from inf to 64.13928, saving model to ./liner_regressor_x_weights.hdf5
1853/1853 - 469s - loss: 114.4416 - val_loss: 64.1393

185300


we are in vad

Epoch 00001: val_loss improved from inf to 19.83136, saving model to ./liner_regressor_y_weights.hdf5
1853/1853 - 544s - loss: 122.4122 - val_loss: 19.8314



In [120]:
# epoch = 3
# TrainERR=hist.history['loss']
# ValidERR=hist.history['val_loss']
# # print('@%f, Minimun error:%f, at iteration: %i' % (hist.history['val_loss'][epoch-1], np.min(np.asarray(ValidERR)),np.argmin(np.asarray(ValidERR))+1))
# print('drawing the training process...')
# plt.figure(2)
# plt.plot(range(1,epoch+1),TrainERR,'b',label='TrainERR')
# plt.plot(range(1,epoch+1),ValidERR,'r',label='ValidERR')
# plt.xlim([1,epoch])
# plt.legend()
# plt.xlabel('epoch')
# plt.ylabel('error')
# plt.grid(True)
# plt.show()

In [155]:
columns = []
columns.append("ID")
for i in range(30):
    num = "v"+str(i+1)
    columns.append(num)
    
df_x = pd.DataFrame(columns=columns)
df_y = pd.DataFrame(columns=columns)

In [156]:
test_list = glob(os.path.join(test_path, '*'))
print(len(test_list))

for x in tqdm(test_list):
    test_x = np.empty((1,19))
    with open(x, 'rb') as f:
        data = pickle.load(f)
        new_row = []
        scene_id = data['scene_idx']
        new_row.append(scene_id)
        agent_id = data['agent_id']
        idx = np.where(data["track_id"] == data["agent_id"])[0][0]   
        input_data = data['p_in'][idx,:,:]
        test_x[0,:] = input_data[:,0]

        test_y = model_x.predict(test_x)

        for i in range(30):
            new_row.append(test_y[0][i])
        df_length = len(df_x)
        df_x.loc[df_length] = new_row

        
        
for x in tqdm(test_list):
    test_x = np.empty((1,19))
    with open(x, 'rb') as f:
        data = pickle.load(f)
        new_row = []
        scene_id = data['scene_idx']
        new_row.append(scene_id)
        agent_id = data['agent_id']
        idx = np.where(data["track_id"] == data["agent_id"])[0][0]   
        input_data = data['p_in'][idx,:,:]
        test_x[0,:] = input_data[:,1]
        test_y = model_y.predict(test_x)

        for i in range(30):
            new_row.append(test_y[0][i])
        df_length = len(df_y)
        df_y.loc[df_length] = new_row

  0%|          | 13/3200 [00:00<00:24, 128.13it/s]

3200


100%|██████████| 3200/3200 [00:14<00:00, 223.95it/s]


In [157]:
df_x['ID'] = df_x['ID'].map(round)
df_y['ID'] = df_y['ID'].map(round)

In [158]:
df_x

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30
0,34189,613.820129,615.590027,615.948303,614.920166,613.891357,613.652588,616.016846,613.597351,615.494263,...,613.275574,615.556396,614.707153,611.868164,615.290283,615.470032,614.801575,615.702454,613.813416,613.901733
1,1454,598.949341,600.664673,601.007874,600.001587,598.999207,598.739624,601.062561,598.693481,600.585510,...,598.370544,600.635132,599.793030,596.993835,600.362854,600.553772,599.873291,600.809448,598.958435,598.986694
2,32404,612.187500,613.969910,614.401001,613.345581,612.339294,612.157593,614.452148,612.113586,613.859009,...,611.797424,613.939453,613.086548,610.335266,613.714844,613.840942,613.241821,614.014587,612.151428,612.341370
3,33991,426.615082,427.856842,428.156982,427.426056,426.719116,426.606567,428.196564,426.570129,427.774231,...,426.353546,427.834900,427.257782,425.342194,427.684967,427.766632,427.362488,427.878082,426.581940,426.740112
4,30427,610.273804,612.028381,612.380310,611.361755,610.337341,610.087097,612.449646,610.033264,611.938416,...,609.715393,611.999329,611.147644,608.315186,611.727356,611.911743,611.232178,612.154541,610.275330,610.338135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,7717,1944.087891,1949.622681,1950.578857,1947.358398,1944.079224,1943.047363,1950.783813,1942.896484,1949.414429,...,1941.853271,1949.532959,1946.729126,1937.465332,1948.539673,1949.277954,1946.850342,1950.279663,1944.208252,1943.976685
3196,5734,2113.733643,2119.950684,2121.424805,2117.772461,2114.279541,2113.790527,2121.645996,2113.622803,2119.501465,...,2112.532471,2119.774170,2116.937500,2107.559326,2119.115967,2119.468262,2117.505615,2119.871094,2113.467041,2114.463867
3197,33829,748.536743,750.685547,751.156921,749.886414,748.639221,748.352234,751.225342,748.271851,750.587830,...,747.896240,750.663757,749.621338,746.148499,750.357910,750.563721,749.710632,750.842957,748.542236,748.629333
3198,4515,599.729431,601.459045,601.827637,600.817017,599.814758,599.599487,601.891113,599.548035,601.358948,...,599.234375,601.426880,600.599548,597.848267,601.183228,601.340576,600.703491,601.552185,599.712891,599.827393


In [159]:
df_y

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30
0,34189,932.539734,931.854187,931.878662,930.050598,930.117737,929.557861,930.042969,931.244202,931.388916,...,929.089478,930.523376,931.207214,930.044983,931.954224,931.775574,931.551575,933.782654,929.981506,929.787231
1,1454,1229.076660,1228.106934,1228.156128,1225.728638,1225.844971,1225.078613,1225.763550,1227.276855,1227.486328,...,1224.486694,1226.358276,1227.236450,1225.736450,1228.223389,1227.987793,1227.681396,1230.626953,1225.598389,1225.361938
2,32404,1523.021851,1521.942505,1521.975464,1519.005493,1519.071777,1518.224731,1518.910889,1520.990723,1521.192871,...,1517.383911,1519.755981,1520.925293,1518.959473,1522.077148,1521.847046,1521.467651,1525.114136,1518.964355,1518.605713
3,33991,1133.557495,1132.818359,1132.840698,1130.644287,1130.650146,1130.077759,1130.501221,1132.167603,1132.278076,...,1129.399536,1131.190674,1132.107300,1130.589844,1132.921509,1132.786011,1132.508057,1135.227783,1130.696289,1130.392334
4,30427,976.622681,975.798767,975.848938,973.904297,974.035522,973.379883,974.009277,975.104187,975.295715,...,972.963074,974.417114,975.074585,973.939697,975.900330,975.677368,975.436707,977.776550,973.734802,973.587952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,7717,482.916840,482.758850,482.754486,481.859253,481.757355,481.640167,481.624481,482.595581,482.565460,...,481.240509,482.064484,482.567627,481.779694,482.819427,482.828156,482.733276,483.899536,482.059906,481.845947
3196,5734,624.122986,623.497314,623.544189,622.274597,622.427368,621.911499,622.440796,622.990662,623.156494,...,621.735413,622.631348,622.980103,622.350525,623.572510,623.364746,623.226807,624.707092,622.076843,622.033875
3197,33829,909.961426,909.396118,909.398804,907.660400,907.628113,907.217957,907.555176,908.874634,909.004639,...,906.635742,908.142395,908.832703,907.638367,909.483459,909.371277,909.117493,911.340942,907.637085,907.469666
3198,4515,1410.751221,1409.646240,1409.695435,1406.914795,1407.053223,1406.179443,1406.931274,1408.707886,1408.934937,...,1405.471191,1407.630371,1408.640991,1406.914062,1409.763794,1409.507324,1409.161377,1412.521729,1406.768799,1406.489746


In [160]:
columns_sub = []
columns_sub.append("ID")
for i in range(60):
    num = "v"+str(i+1)
    columns_sub.append(num)
    
df_sub = pd.DataFrame(columns=columns_sub)

In [162]:
for i in range(df_x.shape[0]):
    row_x = df_x.iloc[i]
    row_y = df_y.iloc[i]
    new_row = []
    new_row.append(row_x[0])
    for i in range(1,31):
        new_row.append(row_x[i])
        new_row.append(row_y[i])
    df_length = len(df_sub)
    df_sub.loc[df_length] = new_row

In [164]:
df_sub['ID'] = df_sub['ID'].map(round)

In [165]:
df_sub

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
0,34189,613.820129,932.539734,615.590027,931.854187,615.948303,931.878662,614.920166,930.050598,613.891357,...,615.470032,931.775574,614.801575,931.551575,615.702454,933.782654,613.813416,929.981506,613.901733,929.787231
1,1454,598.949341,1229.076660,600.664673,1228.106934,601.007874,1228.156128,600.001587,1225.728638,598.999207,...,600.553772,1227.987793,599.873291,1227.681396,600.809448,1230.626953,598.958435,1225.598389,598.986694,1225.361938
2,32404,612.187500,1523.021851,613.969910,1521.942505,614.401001,1521.975464,613.345581,1519.005493,612.339294,...,613.840942,1521.847046,613.241821,1521.467651,614.014587,1525.114136,612.151428,1518.964355,612.341370,1518.605713
3,33991,426.615082,1133.557495,427.856842,1132.818359,428.156982,1132.840698,427.426056,1130.644287,426.719116,...,427.766632,1132.786011,427.362488,1132.508057,427.878082,1135.227783,426.581940,1130.696289,426.740112,1130.392334
4,30427,610.273804,976.622681,612.028381,975.798767,612.380310,975.848938,611.361755,973.904297,610.337341,...,611.911743,975.677368,611.232178,975.436707,612.154541,977.776550,610.275330,973.734802,610.338135,973.587952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,7717,1944.087891,482.916840,1949.622681,482.758850,1950.578857,482.754486,1947.358398,481.859253,1944.079224,...,1949.277954,482.828156,1946.850342,482.733276,1950.279663,483.899536,1944.208252,482.059906,1943.976685,481.845947
3196,5734,2113.733643,624.122986,2119.950684,623.497314,2121.424805,623.544189,2117.772461,622.274597,2114.279541,...,2119.468262,623.364746,2117.505615,623.226807,2119.871094,624.707092,2113.467041,622.076843,2114.463867,622.033875
3197,33829,748.536743,909.961426,750.685547,909.396118,751.156921,909.398804,749.886414,907.660400,748.639221,...,750.563721,909.371277,749.710632,909.117493,750.842957,911.340942,748.542236,907.637085,748.629333,907.469666
3198,4515,599.729431,1410.751221,601.459045,1409.646240,601.827637,1409.695435,600.817017,1406.914795,599.814758,...,601.340576,1409.507324,600.703491,1409.161377,601.552185,1412.521729,599.712891,1406.768799,599.827393,1406.489746


In [166]:
df_sub.to_csv(r'./submissions.csv', index = False, header=True)

In [99]:
columns_test = []
columns_test.append("ID")
for i in range(38):
    num = "v"+str(i+1)
    columns_test.append(num)
    
df_test = pd.DataFrame(columns=columns_test)


for x in test_list:
    with open(x, 'rb') as f:
        data = pickle.load(f)
        new_row = []
        scene_id = data['scene_idx']
        new_row.append(scene_id)
        agent_id = data['agent_id']
        idx = np.where(data["track_id"] == data["agent_id"])[0][0]   
        input_data = data['p_in'][idx,:,:]
        for i in range(38):
            new_row.append(input_data.flatten()[i])
        df_length = len(df_test)
        df_test.loc[df_length] = new_row


In [100]:
df_test

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38
0,34189.0,595.562073,936.528992,595.544006,935.963928,595.649536,936.135559,595.667480,935.907166,595.707275,...,596.247437,933.134216,596.408813,932.617249,596.376892,932.458801,596.398865,932.383728,596.435852,932.086731
1,1454.0,579.748535,1236.775146,579.938965,1236.378784,580.365906,1235.755371,580.250061,1235.585815,580.636536,...,582.202637,1229.476196,582.441162,1228.873901,582.233643,1228.338623,582.358765,1227.714478,582.552185,1227.146851
2,32404.0,596.773865,1526.008545,596.632812,1526.094360,596.576294,1526.078125,596.498291,1526.105225,596.269348,...,593.871521,1525.996704,593.444214,1525.877930,592.983887,1525.745117,592.521118,1525.561646,592.052429,1525.351685
3,33991.0,416.036377,1131.377930,415.903534,1131.828003,415.764526,1132.141724,415.624969,1132.613525,415.485992,...,413.496155,1136.546021,413.229736,1136.827271,412.953491,1137.106201,412.656830,1137.415649,412.440948,1137.703003
4,30427.0,591.568970,986.132019,591.628113,985.414978,591.787476,984.686584,591.843811,984.013428,591.922363,...,592.989380,975.482605,592.937866,974.617676,593.038269,973.842896,593.211975,973.422119,593.310730,972.728210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,7717.0,1875.832397,471.821625,1876.988770,472.846069,1878.359009,473.901123,1879.551514,474.995544,1880.836060,...,1893.428467,486.367584,1894.691895,487.410950,1895.904175,488.400177,1897.126709,489.408905,1898.337891,490.422180
3196,5734.0,2069.424072,635.513184,2068.171631,634.562622,2066.786621,633.569031,2065.407471,632.559387,2064.130371,...,2050.308594,621.376709,2048.978516,620.367310,2047.582764,619.339417,2046.292236,618.321838,2044.849243,617.266418
3197,33829.0,726.628113,909.115906,726.607422,909.657166,726.652161,909.966309,726.634827,909.686401,726.640259,...,727.459900,912.470154,727.510315,912.951233,727.566162,913.033325,727.508850,913.745483,727.478943,914.257568
3198,4515.0,582.553650,1419.214355,582.546326,1418.776367,582.498047,1418.246216,582.581909,1417.869873,582.607849,...,582.132080,1412.069458,582.254089,1410.913574,582.018677,1410.472656,582.125854,1409.070923,582.208496,1408.635132


In [43]:
# with open(test_list[len(test_list)-1], 'rb') as f:
#         data = pickle.load(f)
#         new_row = []
#         scene_id = data['scene_idx']
#         new_row.append(scene_id)
#         agent_id = data['agent_id']
#         idx = np.where(data["track_id"] == data["agent_id"])[0][0]   
#         input_data = data['p_in'][idx,:,:]
# #         print("shape of input data ", input_data.flatten().shape)
#         print(input_data.flatten())

[1940.20263672  536.73956299 1939.49035645  536.15679932 1938.88464355
  535.58843994 1938.56872559  535.48388672 1938.06555176  535.0355835
 1937.41503906  534.53570557 1937.06384277  534.18597412 1936.47070312
  533.70111084 1936.07177734  533.32421875 1935.54748535  532.92150879
 1935.10986328  532.57000732 1934.66650391  532.14715576 1934.26171875
  531.79431152 1933.79650879  531.40020752 1933.44433594  531.04626465
 1933.015625    530.68353271 1932.5402832   530.34326172 1932.20288086
  530.04888916 1931.81359863  529.70751953]
